In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from hmmlearn import hmm
from sklearn.metrics import mean_squared_error

In [7]:
data = pd.read_csv('../input_data/GoogleStocks.csv')
data = data.drop('date',1)
data = data.iloc[1:]
data['avg'] = data[['low', 'high']].mean(axis=1)

In [3]:
def HMM(states, timesteps):
    X1_train = []
    X2_train = []
    y_train = []
    print('States:',states,' Timesteps:',timesteps)
    for i in range(timesteps, 755):
        X1_train.append(scaled_data[i-timesteps:i, 5])
        X2_train.append(scaled_data[i-timesteps:i, 1])
        y_train.append(scaled_data[i, 0])
    X1_train,X2_train, y_train = np.asarray(X1_train) ,np.asarray(X2_train), np.array(y_train)
    X = np.column_stack([X1_train, X2_train])
    
    
    remodel = hmm.GaussianHMM(n_components=states, covariance_type="diag", n_iter=100)
    remodel.fit(X)
    hidden_states = remodel.predict(X)
    expected_returns_and_volumes = np.dot(remodel.transmat_, remodel.means_)
    returns_and_volume_columnwise = list(zip(*expected_returns_and_volumes))
    expected_returns = returns_and_volume_columnwise[0]
    predicted_prices = []
    predicted_volumes = []
    for idx in range(755-timesteps):
        state = hidden_states[idx]
        current_price = scaled_data[idx][1]
        predicted_prices.append(current_price + expected_returns[state])
    mse = mean_squared_error(predicted_prices,y_train)
    return mse

In [5]:
sc = MinMaxScaler(feature_range = (0, 1))
scaled_data = sc.fit_transform(data)
timesteps_list = [20,50,75]
states_list = [4,8,12]
combination = 1
score_list = []
for states in states_list:
    for timesteps in timesteps_list:
        print('Combination:',combination)
        score = HMM(states,timesteps)
        score_list.append(score)
        print('Loss:',score,'\n')
        combination += 1

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Combination: 1
States: 4  Timesteps: 20
Loss: 0.06978685757351262 

Combination: 2
States: 4  Timesteps: 50
Loss: 0.08303790607633037 

Combination: 3
States: 4  Timesteps: 75
Loss: 0.09794579452023434 

Combination: 4
States: 8  Timesteps: 20
Loss: 0.06611711533137804 

Combination: 5
States: 8  Timesteps: 50
Loss: 0.0845509518797185 

Combination: 6
States: 8  Timesteps: 75
Loss: 0.10282508017519815 

Combination: 7
States: 12  Timesteps: 20
Loss: 0.06606627658817033 

Combination: 8
States: 12  Timesteps: 50
Loss: 0.08617329208988264 

Combination: 9
States: 12  Timesteps: 75
Loss: 0.10218136692806153 

